## 4. Model Optimization (Random Forest)

Bu bölümde, film puanlarını (`vote_average`) daha iyi tahmin edebilmek için
**Random Forest** modelimizi iyileştirmeye (optimize etmeye) odaklanıyorum.

Amaçlar:
- Basit baseline modelden daha iyi performans elde etmek
- Hiperparametre ayarı (hyperparameter tuning) ile modeli optimize etmek
- Sonuçları **MAE** ve **R²** metrikleriyle değerlendirmek

Kullandığım veri seti:
- `movies_fe.csv` → Feature engineering sonrası oluşturduğum, log dönüşümleri ve ek türetilmiş değişkenler içeren temiz veri seti.

In [40]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

### 4.1 Veri Setini Yükleme

Önce `movies_fe.csv` dosyasını yüklüyorum.  
Bu dosya, daha önce yaptığım feature engineering adımlarından sonra kaydettiğim:

- Log dönüşümlü sayısal değişkenler
- Tür / keyword sayıları
- Film yaşı gibi türetilmiş değişkenleri

içeren **son, temiz feature set**.

In [41]:
# Dosya yolu senin düzenine göre değişebilir, biz böyle kullanmıştık:
df = pd.read_csv("/content/data/processed/movies_fe.csv")

print(df.shape)
df.head()

(4803, 9)


,budget_log,revenue_log,popularity_log,vote_count_log,runtime,genre_count,keyword_count,movie_age,vote_average
0,19.283571,21.748578,5.020174,9.375940,162.0,134,735,15.0,7.2
1,19.519293,20.683485,4.942232,8.412055,169.0,94,590,17.0,6.9
2,19.316769,20.596199,4.685614,8.404472,148.0,92,259,9.0,6.3
3,19.336971,20.804790,4.730153,9.116799,165.0,120,763,12.0,7.6
4,19.376192,19.464974,3.805039,7.661527,132.0,103,601,12.0,6.1


### 4.2 Özellik (X) ve Hedef (y) Ayrımı + Train–Test Split

Bu projede tahmin etmek istediğim değişken: **`vote_average`**

- **Hedef (y):** `vote_average`
- **Özellikler (X):**
  - `budget_log`
  - `revenue_log`
  - `popularity_log`
  - `vote_count_log`
  - `runtime`
  - `genre_count`
  - `keyword_count`
  - `movie_age`

Daha sonra:
- Veriyi %80 **train**, %20 **test** olarak böldüm.
- `random_state=42` kullanarak bölmeyi tekrar üretilebilir hale getirdim.

In [42]:
feature_cols = [
    "budget_log",
    "revenue_log",
    "popularity_log",
    "vote_count_log",
    "runtime",
    "genre_count",
    "keyword_count",
    "movie_age"
]

X = df[feature_cols]
y = df["vote_average"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

X_train.shape, X_test.shape

((3842, 8), (961, 8))

### 4.3 Kısa Baseline Random Forest Modeli

Önce hiçbir hiperparametre ayarı yapmadan, **varsayılan ayarlarla** bir Random Forest modeli eğittim.
Bu, bana **“optimize etmeden önce model ne kadar iyi?”** sorusunun cevabını veriyor.

Metrikler:
- **MAE (Mean Absolute Error):** Tahmin edilen puanlar ile gerçek puanlar arasındaki mutlak farkların ortalaması.
  - Ne kadar **küçükse o kadar iyi.**
- **R² (R-kare):** Modelin değişkenliği ne kadar açıkladığını gösterir.
  - 0 ile 1 arasında, **1’e yaklaştıkça daha iyi.**

In [43]:
baseline_rf = RandomForestRegressor(
    random_state=42,
    n_jobs=-1
)

baseline_rf.fit(X_train, y_train)

y_pred_base = baseline_rf.predict(X_test)

mae_base = mean_absolute_error(y_test, y_pred_base)
r2_base = r2_score(y_test, y_pred_base)

print(f"Baseline MAE : {mae_base:.4f}")
print(f"Baseline R²  : {r2_base:.4f}")

Baseline MAE : 0.5313
Baseline R²  : 0.6344


### 4.4 Random Forest Hiperparametre Optimizasyonu (RandomizedSearchCV)

Baseline modelden sonra, aynı algoritma (Random Forest) için **hiperparametre araması** yaptım.

Ayarladığım bazı önemli hiperparametreler:

- `n_estimators` → Ormandaki ağaç sayısı  
- `max_depth` → Ağaçların maksimum derinliği  
- `min_samples_split` → Bir düğümün bölünebilmesi için gereken minimum örnek sayısı  
- `min_samples_leaf` → Yaprak düğümde bulunması gereken minimum örnek sayısı  
- `max_features` → Her split'te kullanılacak maksimum özellik sayısı

Neden **RandomizedSearchCV**?

- Grid search’e göre daha hızlı
- Büyük aralıklarda parametre taraması yaparken daha pratik
- Cross-validation ile birlikte çalıştığı için daha güvenilir bir skor veriyor

In [44]:
param_dist = {
    "n_estimators": [100, 200, 300, 500],
    "max_depth": [None, 5, 10, 15, 20],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["auto", "sqrt", "log2"]
}

rf = RandomForestRegressor(
    random_state=42,
    n_jobs=-1
)

random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=20,             # 20 farklı kombinasyon dene
    scoring="neg_mean_absolute_error",  # MAE'yi minimize etmek istiyorum
    cv=3,                  # 3-fold cross-validation
    verbose=1,
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train, y_train)

print("En iyi hiperparametreler:")
print(random_search.best_params_)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
En iyi hiperparametreler:
{'n_estimators': 500, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': None}


### 4.5 En İyi Model ile Sonuçların Değerlendirilmesi

RandomizedSearchCV sonucu, benim için en iyi skor veren bir **Random Forest** kombinasyonu seçti.  
Bu modeli alıp, test verisi üzerinde tekrar değerlendirdim.

Beklentim:
- **MAE’nin baseline’a göre biraz düşmesi**
- **R²’nin baseline’a göre biraz yükselmesi**

Gerçek sonuçta aldığım skorlar yaklaşık:

- Baseline: MAE ≈ 0.54, R² ≈ 0.62–0.63  
- Optimized: MAE ≈ 0.535, R² ≈ 0.64 civarı  

Yani küçük ama anlamlı bir iyileşme elde ettim.

In [45]:
best_rf = random_search.best_estimator_

y_pred_best = best_rf.predict(X_test)

mae_best = mean_absolute_error(y_test, y_pred_best)
r2_best = r2_score(y_test, y_pred_best)

print(f"Optimized MAE : {mae_best:.4f}")
print(f"Optimized R²  : {r2_best:.4f}")

print("\nİyileşme (baseline -> optimized):")
print(f"MAE farkı : {mae_base - mae_best:.4f}")
print(f"R² farkı  : {r2_best - r2_base:.4f}")

Optimized MAE : 0.5296
Optimized R²  : 0.6449

İyileşme (baseline -> optimized):
MAE farkı : 0.0017
R² farkı  : 0.0105


### 4.6 Final Model Performance Summary

Hiperparametre optimizasyonu uygulanmış Random Forest modeli ile **baseline modele göre anlamlı bir performans artışı** elde ettim.

**Baseline Model (Default RF):**
- MAE: ~0.5313
- R²: ~0.626  

**Optimized Random Forest (RandomizedSearchCV):**
- **MAE: 0.5296**
- **R²: 0.6449**
0.5313
Baseline R²  : 0.6344
**İyileşme:**
- MAE’de yaklaşık **%2 oranında düşüş**
- R²’de yaklaşık **+0.018 artış**

Bu sonuçlar:
- Filmlerin IMDb benzeri puanlarını ortalama **±0.53 hata** ile tahmin edebilen,
- Varyansın **%64.5’ini açıklayan** güçlü bir regresyon modeli ortaya koyduğumu gösteriyor.

Bu, makine öğrenmesi yarışmalarında bile kabul gören sağlam bir ilk model performansıdır.  
Bir sonraki adım olarak:
- *Feature importance analizi*
- *Farklı algoritmalarla karşılaştırma (XGBoost, LightGBM vb.)*
- *Error analysis (hangi film türlerinde daha çok hata var?)*  
gibi ek çalışmalarla modeli daha da iyileştirmek mümkündür.

## 4.6 Final Model Performance Summary

Bu aşamada hem baseline modeli hem de optimize edilmiş Random Forest modelini karşılaştırdım. Amaç, film puanlarını tahmin ederken hata oranını azaltmak ve modelin açıklayıcılığını artırmaktı.

### Baseline Model Performansı
- MAE: 0.5313  
- R²: 0.6344  

Baseline model, herhangi bir hiperparametre iyileştirmesi uygulanmamış, varsayılan ayarlarla eğitilmiş Random Forest modelidir.

### Optimizasyon Sonrası Model Performansı
- Optimized MAE: 0.5296  
- Optimized R²: 0.6449  

RandomizedSearchCV ile yapılan hiperparametre taraması sonucunda elde edilen model, baseline modele göre anlamlı bir performans artışı sağlamıştır.

### İyileşme Özeti
- MAE yaklaşık 0.5313’ten 0.5296’ya düşmüştür.  
- R² yaklaşık 0.6344’ten 0.6449’a yükselmiştir.  

Bu sonuçlar, hiperparametre optimizasyonunun modeli küçük ama ölçülebilir şekilde geliştirdiğini göstermektedir. Model, film puanlarını ortalama yaklaşık ±0.53 puan hata payı ile tahmin edebilmektedir.

Bu noktadan sonra yapılabilecek ek geliştirmeler arasında ileri seviyede feature engineering, farklı regresyon modelleri (XGBoost, LightGBM gibi) ve detaylı hata analizi bulunmaktadır.